In [8]:
#import get to call a get request on the site
from requests import get

#get the first page of the east bay housing prices
response = get('https://philadelphia.craigslist.org/d/apartments-housing-for-rent/search/apa') #get rid of those lame-o's that post a housing option without a pic using their filter

from bs4 import BeautifulSoup
html_soup = BeautifulSoup(response.text, 'html.parser')

#get the macro-container for the housing posts
#posts = html_soup.find_all('p', class_= 'results-row')
#print(type(posts)) #to double check that I got a ResultSet
#print(len(posts)) #to double check I got 120 (elements/page)

In [9]:
#posts = html_soup.find_all('div', class_ = 'mapAndAttrs')

In [13]:
#build out the loop
from time import sleep
import re
from random import randint #avoid throttling by not sending too many requests one after the other
from warnings import warn
from time import time
from IPython.core.display import clear_output
import numpy as np

#find the total number of posts to find the limit of the pagination
results_num = html_soup.find('div', class_= 'search-legend')
results_total = int(results_num.find('span', class_= 'totalcount').text) #pulled the total count of posts as the upper bound of the pages array

#each page has 119 posts so each new page is defined as follows: s=120, s=240, s=360, and so on. So we need to step in size 120 in the np.arange function
pages = np.arange(0, results_total+1, 120)

iterations = 0

post_timing = [ ]
post_hoods = []
post_title_texts = []
bedroom_counts = []
sqfts = []
post_links = []
post_prices = []

for page in pages:
    
    #get request
    response = get("https://philadelphia.craigslist.org/d/apartments-housing-for-rent/search/apa" 
                   + "s=" #the parameter for defining the page number 
                   + str(page) #the page number in the pages array from earlier
                   + "&hasPic=1"
                   + "&availabilityMode=0")

    sleep(randint(1,5))
     
    #throw warning for status codes that are not 200
    if response.status_code != 200:
        warn('Request: {}; Status code: {}'.format(requests, response.status_code))
        
    #define the html text
    page_html = BeautifulSoup(response.text, 'html.parser')
    
    #define the posts
    posts = html_soup.find_all('li', class_= 'result-row')
        
    #extract data item-wise
    for post in posts:

        if post.find('span', class_ = 'result-hood') is not None:

            #posting date
            #grab the datetime element 0 for date and 1 for time
            post_datetime = post.find('time', class_= 'result-date')['datetime']
            post_timing.append(post_datetime)

            #neighborhoods
            post_hood = post.find('span', class_= 'result-hood').text
            post_hoods.append(post_hood)

            #title text
            post_title = post.find('a', class_='result-title hdrlnk')
            post_title_text = post_title.text
            post_title_texts.append(post_title_text)

            #post link
            post_link = post_title['href']
            post_links.append(post_link)
            
            #removes the \n whitespace from each side, removes the currency symbol, and turns it into an int
            
            post_price= post.a.text.strip().replace("$", "").replace(",", "")
            if post_price == '':
                post_price == np.nan
                post_prices.append(post_price)
            else:
                post_prices.append(post_price)
            
            if post.find('span', class_ = 'housing') is not None:
                
                #if the first element is accidentally square footage
                if 'ft2' in post.find('span', class_ = 'housing').text.split()[0]:
                    
                    #make bedroom nan
                    bedroom_count = np.nan
                    bedroom_counts.append(bedroom_count)
                    
                    #make sqft the first element
                    sqft = int(post.find('span', class_ = 'housing').text.split()[0][:-3])
                    sqfts.append(sqft)
                    
                #if the length of the housing details element is more than 2
                elif len(post.find('span', class_ = 'housing').text.split()) > 2:
                    
                    #therefore element 0 will be bedroom count
                    bedroom_count = post.find('span', class_ = 'housing').text.replace("br", "").split()[0]
                    bedroom_counts.append(bedroom_count)
                    
                    #and sqft will be number 3, so set these here and append
                    sqft = int(post.find('span', class_ = 'housing').text.split()[2][:-3])
                    sqfts.append(sqft)
                    
                #if there is num bedrooms but no sqft
                elif len(post.find('span', class_ = 'housing').text.split()) == 2:
                    
                    #therefore element 0 will be bedroom count
                    bedroom_count = post.find('span', class_ = 'housing').text.replace("br", "").split()[0]
                    bedroom_counts.append(bedroom_count)
                    
                    #and sqft will be number 3, so set these here and append
                    sqft = np.nan
                    sqfts.append(sqft)                    
                
                else:
                    bedroom_count = np.nan
                    bedroom_counts.append(bedroom_count)
                
                    sqft = np.nan
                    sqfts.append(sqft)
                
            #if none of those conditions catch, make bedroom nan, this won't be needed    
            else:
                bedroom_count = np.nan
                bedroom_counts.append(bedroom_count)
                
                sqft = np.nan
                sqfts.append(sqft)
            #    bedroom_counts.append(bedroom_count)
                
            #    sqft = np.nan
            #    sqfts.append(sqft)
                
    iterations += 1
    print("Page " + str(iterations) + " scraped successfully!")

print("\n")

print("Scrape complete!")

Page 1 scraped successfully!
Page 2 scraped successfully!
Page 3 scraped successfully!
Page 4 scraped successfully!
Page 5 scraped successfully!
Page 6 scraped successfully!
Page 7 scraped successfully!
Page 8 scraped successfully!
Page 9 scraped successfully!
Page 10 scraped successfully!
Page 11 scraped successfully!
Page 12 scraped successfully!
Page 13 scraped successfully!
Page 14 scraped successfully!
Page 15 scraped successfully!
Page 16 scraped successfully!
Page 17 scraped successfully!
Page 18 scraped successfully!
Page 19 scraped successfully!
Page 20 scraped successfully!
Page 21 scraped successfully!
Page 22 scraped successfully!
Page 23 scraped successfully!
Page 24 scraped successfully!
Page 25 scraped successfully!
Page 26 scraped successfully!


Scrape complete!


In [14]:
import pandas as pd

In [15]:
t = np.column_stack((post_prices, sqfts, post_links,bedroom_counts,post_title_texts, post_hoods,post_timing))

In [16]:
philly_craiglist_housing = pd.DataFrame(t)


In [17]:
philly_craiglist_housing.columns = ['post_prices', 'sqfts', 'post_links','bedroom_counts', 'post_title_texts', 'post_hoods','post_timing']

In [18]:
philly_craiglist_housing

,post_prices,sqfts,post_links,bedroom_counts,post_title_texts,post_hoods,post_timing
0,595,nan,https://philadelphia.craigslist.org/apa/d/priv...,5,PRIVATE OWNED-CURRENTLY 2-6 BEDRM UNITS AVAIL ...,(PETS OK-GARAGES-LARGE YARDS),2021-02-14 19:41
1,1595,1600.0,https://philadelphia.craigslist.org/apa/d/phil...,2,*QueenVillage Luxury 2BR 2Bath Huge Roof Deck!...,(Queen Village - S. 2nd/Fitzwater St),2021-02-14 19:37
2,,nan,https://philadelphia.craigslist.org/apa/d/vall...,1,$730 1 BR 1BATH KING OF PRUSSIA - IN UNIT W/D ...,(19406),2021-02-14 19:36
3,2200,1400.0,https://philadelphia.craigslist.org/apa/d/phil...,4,****4BD*3BA**NEAR TEMPLE U****AUG. 2021****,(Philadelphia),2021-02-14 19:30
4,1750,1008.0,https://philadelphia.craigslist.org/apa/d/phil...,2,"Nice, spacious 2 bedroom apt in Great Location",(Washington Square West),2021-02-14 19:22
...,...,...,...,...,...,...,...
2309,1500,nan,https://philadelphia.craigslist.org/apa/d/phil...,4,Renovated 4-BR 2-Bath Apt. with Large Yard Rig...,(Temple Univ 2148-50 N Carlisle Apt 1),2021-02-14 15:22
2310,850,870.0,https://philadelphia.craigslist.org/apa/d/phil...,2,50th St & Race- 2 BD 1 Bath 3rd Floor Apt- Wal...,"(141 N 50th Street, Apt 3)",2021-02-14 15:22
2311,2300,1808.0,https://philadelphia.craigslist.org/apa/d/phil...,4,House with 4 Lrg Bed 2 Lrg Bath walking distan...,(3224 Mt. Vernon Street),2021-02-14 15:22
2312,1095,900.0,https://philadelphia.craigslist.org/apa/d/phil...,2,Short Term Rental 2 Bed 1 Bath - 1 Block From ...,(1861 N Willington Street Unit 1),2021-02-14 15:21


In [19]:
philly_craiglist_housing.to_csv('craiglist_housing_price.csv')

In [20]:
philly_craiglist_housing

,post_prices,sqfts,post_links,bedroom_counts,post_title_texts,post_hoods,post_timing
0,595,nan,https://philadelphia.craigslist.org/apa/d/priv...,5,PRIVATE OWNED-CURRENTLY 2-6 BEDRM UNITS AVAIL ...,(PETS OK-GARAGES-LARGE YARDS),2021-02-14 19:41
1,1595,1600.0,https://philadelphia.craigslist.org/apa/d/phil...,2,*QueenVillage Luxury 2BR 2Bath Huge Roof Deck!...,(Queen Village - S. 2nd/Fitzwater St),2021-02-14 19:37
2,,nan,https://philadelphia.craigslist.org/apa/d/vall...,1,$730 1 BR 1BATH KING OF PRUSSIA - IN UNIT W/D ...,(19406),2021-02-14 19:36
3,2200,1400.0,https://philadelphia.craigslist.org/apa/d/phil...,4,****4BD*3BA**NEAR TEMPLE U****AUG. 2021****,(Philadelphia),2021-02-14 19:30
4,1750,1008.0,https://philadelphia.craigslist.org/apa/d/phil...,2,"Nice, spacious 2 bedroom apt in Great Location",(Washington Square West),2021-02-14 19:22
...,...,...,...,...,...,...,...
2309,1500,nan,https://philadelphia.craigslist.org/apa/d/phil...,4,Renovated 4-BR 2-Bath Apt. with Large Yard Rig...,(Temple Univ 2148-50 N Carlisle Apt 1),2021-02-14 15:22
2310,850,870.0,https://philadelphia.craigslist.org/apa/d/phil...,2,50th St & Race- 2 BD 1 Bath 3rd Floor Apt- Wal...,"(141 N 50th Street, Apt 3)",2021-02-14 15:22
2311,2300,1808.0,https://philadelphia.craigslist.org/apa/d/phil...,4,House with 4 Lrg Bed 2 Lrg Bath walking distan...,(3224 Mt. Vernon Street),2021-02-14 15:22
2312,1095,900.0,https://philadelphia.craigslist.org/apa/d/phil...,2,Short Term Rental 2 Bed 1 Bath - 1 Block From ...,(1861 N Willington Street Unit 1),2021-02-14 15:21


In [21]:
post_link =post_links[2:6]

In [22]:
philly_craiglist_housing.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 2314 entries, 0 to 2313
Data columns (total 7 columns):
 #   Column            Non-Null Count  Dtype 
---  ------            --------------  ----- 
 0   post_prices       2314 non-null   object
 1   sqfts             2314 non-null   object
 2   post_links        2314 non-null   object
 3   bedroom_counts    2314 non-null   object
 4   post_title_texts  2314 non-null   object
 5   post_hoods        2314 non-null   object
 6   post_timing       2314 non-null   object
dtypes: object(7)
memory usage: 126.7+ KB


In [23]:
len(philly_craiglist_housing[philly_craiglist_housing['post_prices']== 'nan'])

0

In [24]:
philly_craiglist_housing.isnull().sum()

post_prices         0
sqfts               0
post_links          0
bedroom_counts      0
post_title_texts    0
post_hoods          0
post_timing         0
dtype: int64

In [25]:
philly_craiglist_housing['post_links'][2]

'https://philadelphia.craigslist.org/apa/d/valley-forge-br-1bath-king-of-prussia/7273669856.html'

In [266]:
post_link= philly_craiglist_housing['post_links']

In [36]:
import re

In [30]:
post_link =post_links
post_link

['https://philadelphia.craigslist.org/apa/d/private-owned-currently-6-bedrm-units/7277555753.html',
 'https://philadelphia.craigslist.org/apa/d/philadelphia-queenvillage-luxury-2br/7277552399.html',
 'https://philadelphia.craigslist.org/apa/d/valley-forge-br-1bath-king-of-prussia/7273669856.html',
 'https://philadelphia.craigslist.org/apa/d/philadelphia-4bd3banear-temple-uaug-2021/7277552717.html',
 'https://philadelphia.craigslist.org/apa/d/philadelphia-nice-spacious-bedroom-apt/7272929110.html',
 'https://philadelphia.craigslist.org/apa/d/west-chester-two-bedroom-two-full-bath/7269472193.html',
 'https://philadelphia.craigslist.org/apa/d/west-chester-one-bedroom-one-bath-a-den/7275573738.html',
 'https://philadelphia.craigslist.org/apa/d/philadelphia-bed-52-baths-car-gar/7273488092.html',
 'https://philadelphia.craigslist.org/apa/d/west-chester-resort-style-swimming-pool/7275696805.html',
 'https://philadelphia.craigslist.org/apa/d/philadelphia-northeast-fox-chase-duplex/7277534736.h

In [267]:
No_of_bathrooms = []
No_of_bedrooms = []
latitude = []
longitude = []
pet=[]
dog = []
laundry =[]
parking =[]
floor = []
Apartment_type = []
address = []
price = []
smoking = []

#for i in philly_craiglist_housing['post_links']:
for i in post_link:
    indi_response = get(i)
     
    #define the html text
    page_html = BeautifulSoup(indi_response.text, 'html.parser')

    #define the posts
    ind_posts = page_html.find_all('div', class_ = 'mapAndAttrs')
    for links in ind_posts:
        
        #the bedroom , bathroom and space
        if links.find('p', class_ = 'attrgroup') is not None:
            
            apartment_phys = links.find('p', class_= 'attrgroup')

        
        #bedroom -bathroom
            No_of_bedrooms.append(apartment_phys.span.text.split('/')[0])
            No_of_bathrooms.append(apartment_phys.span.text.split('/')[1])
        
        #latitude and longitude
        long_lat = links.find('div', class_ = "viewposting")
        longitude.append(float(long_lat['data-longitude']))
        latitude.append(float(long_lat['data-latitude']))
        
        #address
        #add = links.find('div', class_ = 'mapaddress').text
        #address.append(add)
        
        #Other _features
        
        if links.find_all('p', class_ = 'attrgroup')[1] is not None:
            t = links.find_all('p', class_ = 'attrgroup')[1]
            
            span=t.text.split('\n')
            
            if 'cats are OK - purrr' in span:
                pet.append('cats are OK - purrr')
            elif 'dogs are OK - wooof':
                pet.append('dogs are OK - wooof')
            else:
                pet.append('no pet')
                
        
            #laundry
            if 'laundry in bldg' in span:
                laundry.append('laundry in bld')
            elif 'w/d in unit' in span:
                laundry.append('w/d in unit')
            elif 'laundry on site' in span:
                laundry.append('laundry on site')
            elif 'no laundry on site' in span:
                laundry.append('no laundry on site')
            elif 'w/d hookups' in span:
                laundry.append('w/d hookups')
            else:
                laundry.append(np.nan)
                
            #house
            if 'house' in span:
                Apartment_type.append('house')
            elif 'apartment' in span:
                Apartment_type.append('apartment')
            elif 'townhouse' in span:
                Apartment_type.append('townhouse')
            elif 'condo' in span:
                Apartment_type.append('condo')
            elif 'duplex' in span:
                Apartment_type.append('duplex')
            else:
                Apartment_type.append(np.nan)
            
            #floor
            if 'flooring: carpet' in span:
                floor.append('flooring: carpet')
            elif 'flooring: wood' in span:
                floor.append('flooring: wood')
            else:
                floor.append(np.nan)
                

        
            #Parking
            if 'no parking' in span:
                parking.append('no parking')
            elif 'off-street parking' in span:
                parking.append('off-street parking')
            elif 'street parking' in span:
                parking.append('street parking')
            elif 'detached garage' in span:
                parking.append('detached garage')
            elif 'attached garage' in span:
                parking.append('attached garage')
            else:
                parking.append('np.nan')
            
            #smoking
            if 'smoking' in span:
                smoking.append('smoking')
            elif 'no smoking' in span:
                smoking.append('no smoking')
            else:
                smoking.append(np.nan)
            

In [268]:
#create the numpy array
df_np = np.stack((No_of_bathrooms,No_of_bedrooms, latitude, longitude, pet, laundry, parking, floor, Apartment_type, smoking), axis=-1)

In [273]:
df_np[2000]

array([' 1Ba', '1BR ', '39.976547', '-75.210857', 'cats are OK - purrr',
       'w/d in unit', 'attached garage', 'nan', 'apartment', 'no smoking'],
      dtype='<U32')

In [269]:
col_names = 'No_of_bathrooms', 'No_of_bedrooms', 'latitude', 'longitude', 'pet', 'laundry', 'parking', 'floor', 'Apartment_type', 'smoking'

In [274]:
#create the dataframe from numpy arrays
df = pd.DataFrame(data=df_np, columns= col_names)

In [275]:
df

,No_of_bathrooms,No_of_bedrooms,latitude,longitude,pet,laundry,parking,floor,Apartment_type,smoking
0,2Ba,5BR,40.1238,-75.1148,cats are OK - purrr,w/d in unit,detached garage,nan,house,nan
1,2Ba,2BR,39.938205,-75.14657,dogs are OK - wooof,laundry in bld,street parking,nan,apartment,no smoking
2,1Ba,1BR,40.104558,-75.416913,cats are OK - purrr,w/d in unit,np.nan,nan,apartment,nan
3,3Ba,4BR,39.9811,-75.174,cats are OK - purrr,w/d in unit,street parking,flooring: wood,apartment,no smoking
4,1Ba,2BR,39.9362,-75.1563,cats are OK - purrr,w/d in unit,street parking,flooring: wood,condo,nan
...,...,...,...,...,...,...,...,...,...,...
2309,2Ba,4BR,39.986492,-75.157899,cats are OK - purrr,laundry in bld,street parking,nan,apartment,no smoking
2310,1Ba,2BR,39.9626,-75.220271,cats are OK - purrr,laundry in bld,street parking,flooring: wood,apartment,no smoking
2311,2Ba,4BR,39.964814,-75.190265,cats are OK - purrr,w/d in unit,street parking,flooring: wood,house,no smoking
2312,1Ba,2BR,39.981552,-75.161401,cats are OK - purrr,laundry in bld,street parking,nan,apartment,no smoking


In [276]:
df.to_csv('df_crailist_housing_web_scrap.csv')

In [277]:
#combine the two dataframe
df_combo = pd.concat([df, philly_craiglist_housing], axis=1)

In [278]:
df_combo

,No_of_bathrooms,No_of_bedrooms,latitude,longitude,pet,laundry,parking,floor,Apartment_type,smoking,post_prices,sqfts,post_links,bedroom_counts,post_title_texts,post_hoods,post_timing
0,2Ba,5BR,40.1238,-75.1148,cats are OK - purrr,w/d in unit,detached garage,nan,house,nan,595,nan,https://philadelphia.craigslist.org/apa/d/priv...,5,PRIVATE OWNED-CURRENTLY 2-6 BEDRM UNITS AVAIL ...,(PETS OK-GARAGES-LARGE YARDS),2021-02-14 19:41
1,2Ba,2BR,39.938205,-75.14657,dogs are OK - wooof,laundry in bld,street parking,nan,apartment,no smoking,1595,1600.0,https://philadelphia.craigslist.org/apa/d/phil...,2,*QueenVillage Luxury 2BR 2Bath Huge Roof Deck!...,(Queen Village - S. 2nd/Fitzwater St),2021-02-14 19:37
2,1Ba,1BR,40.104558,-75.416913,cats are OK - purrr,w/d in unit,np.nan,nan,apartment,nan,,nan,https://philadelphia.craigslist.org/apa/d/vall...,1,$730 1 BR 1BATH KING OF PRUSSIA - IN UNIT W/D ...,(19406),2021-02-14 19:36
3,3Ba,4BR,39.9811,-75.174,cats are OK - purrr,w/d in unit,street parking,flooring: wood,apartment,no smoking,2200,1400.0,https://philadelphia.craigslist.org/apa/d/phil...,4,****4BD*3BA**NEAR TEMPLE U****AUG. 2021****,(Philadelphia),2021-02-14 19:30
4,1Ba,2BR,39.9362,-75.1563,cats are OK - purrr,w/d in unit,street parking,flooring: wood,condo,nan,1750,1008.0,https://philadelphia.craigslist.org/apa/d/phil...,2,"Nice, spacious 2 bedroom apt in Great Location",(Washington Square West),2021-02-14 19:22
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2309,2Ba,4BR,39.986492,-75.157899,cats are OK - purrr,laundry in bld,street parking,nan,apartment,no smoking,1500,nan,https://philadelphia.craigslist.org/apa/d/phil...,4,Renovated 4-BR 2-Bath Apt. with Large Yard Rig...,(Temple Univ 2148-50 N Carlisle Apt 1),2021-02-14 15:22
2310,1Ba,2BR,39.9626,-75.220271,cats are OK - purrr,laundry in bld,street parking,flooring: wood,apartment,no smoking,850,870.0,https://philadelphia.craigslist.org/apa/d/phil...,2,50th St & Race- 2 BD 1 Bath 3rd Floor Apt- Wal...,"(141 N 50th Street, Apt 3)",2021-02-14 15:22
2311,2Ba,4BR,39.964814,-75.190265,cats are OK - purrr,w/d in unit,street parking,flooring: wood,house,no smoking,2300,1808.0,https://philadelphia.craigslist.org/apa/d/phil...,4,House with 4 Lrg Bed 2 Lrg Bath walking distan...,(3224 Mt. Vernon Street),2021-02-14 15:22
2312,1Ba,2BR,39.981552,-75.161401,cats are OK - purrr,laundry in bld,street parking,nan,apartment,no smoking,1095,900.0,https://philadelphia.craigslist.org/apa/d/phil...,2,Short Term Rental 2 Bed 1 Bath - 1 Block From ...,(1861 N Willington Street Unit 1),2021-02-14 15:21


In [279]:
df_combo.to_csv('web_scrap_craiglist_data.csv')